# Read test radar test file and define gridding and interpolation scheme

In [ ]:
import xarray as xr
from open_radar_data import DATASETS
import xradar as xd
import matplotlib.pyplot as plt
import numpy as np

import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import src.global_variables as global_vars
import src.local_variables as local_vars

**Read the radar data**

In [ ]:
# data_dir = local_vars.radar_file_dir
# filename = data_dir + '/2024/06/01/rainbow5/HUR/240km_12ele_DP_PROD011.vol/2024060112000700dBZ.vol'
filename = "../data/01-06-2024/2024060112000700dBZ.vol"

In [ ]:
def load_all_sweeps(filename):
    """Load all 12 sweeps for one parameter"""
    sweeps = []
    for i in range(12):
        ds = xr.open_dataset(filename, group=f"sweep_{i}", engine="rainbow")
        sweeps.append(ds)
    return sweeps

sweeps = load_all_sweeps(filename)

**Plot lowest elevation**

In [ ]:
range_vals = sweeps[0].range.values      # shape (480,)
azimuth_vals = sweeps[0].azimuth.values  # shape (360,) 
dbzh_vals = sweeps[0].DBZH.values        # shape (360, 480)

# Colormap limits
vmin, vmax = -60, 60

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Manual plot with imshow
im1 = ax1.imshow(dbzh_vals, 
                 extent=[range_vals.min(), range_vals.max(), 
                        azimuth_vals.min(), azimuth_vals.max()],
                 aspect='auto', 
                 origin='lower',
                 cmap='seismic',
                 vmin=vmin,
                 vmax=vmax)
ax1.set_xlabel('Range (m)')
ax1.set_ylabel('Azimuth (degrees)')
ax1.set_title('Manual imshow plot')
plt.colorbar(im1, ax=ax1, label='DBZH (dBZ)')

# Built-in xarray plot for comparison
sweeps[0].DBZH.plot(ax=ax2)
ax2.set_title('Built-in xarray plot')

plt.tight_layout()
plt.show()

**Convert from polar to Cartesian coordinates**

In [ ]:
def spherical_to_cartesian_3D(sweep):
    """Convert radar field from spherical to 3D Cartesian coordinates"""

    range_vals = sweep.range.values
    azimuth_vals = sweep.azimuth.values
    elevation_angle = sweep.sweep_fixed_angle.values

    R_mesh, Az_mesh = np.meshgrid(range_vals, azimuth_vals)

    phi = np.pi/2 - np.radians(elevation_angle) # Convert to zenith angle
    theta = np.radians(Az_mesh)

    x = R_mesh * np.sin(phi) * np.sin(theta)
    y = R_mesh * np.sin(phi) * np.cos(theta)
    z = R_mesh * np.cos(phi)

    return x, y, z

x, y, z = spherical_to_cartesian_3D(sweeps[0])

In [ ]:
# Plot top view to verify correctness
fig, ax = plt.subplots(1, 2, figsize=(15, 6))  
# Top view (X-Y plane)
scatter1 = ax[0].scatter(x.flatten()/1000, y.flatten()/1000,
                     c=sweeps[0].DBZH.values.flatten(), s=0.5, cmap="seismic", vmin=vmin, vmax=vmax)
ax[0].set_xlabel('X (km)')
ax[0].set_ylabel('Y (km)')
ax[0].set_title('Manual view')
ax[0].set_aspect('equal')
# ax[0].set_xlim([-40,40])
# ax[0].set_ylim([-40, 40])

# Add colorbar
cbar = plt.colorbar(scatter1, ax=ax[0])  
cbar.set_label('DBZH (dBZ)')

# Verify with built-in plotting routine
rd = sweeps[0].xradar.georeference()
rd.DBZH.plot(ax=ax[1], x="x", y="y")

plt.show()

In [ ]:
# Plot just the coordinate points to see the cone shape
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Sample every 10th point to reduce clutter
step = 10
ax.scatter(x[::step, ::step]/1000, y[::step, ::step]/1000, z[::step, ::step]/1000, s=1)

ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
ax.set_zlabel('Z (km)')
ax.set_title('Radar Cone Structure')
plt.show()

In [ ]:
# Plot radar cone with radar data
from mpl_toolkits.mplot3d import Axes3D

# Flatten arrays for scatter plot
x_flat = x.flatten()
y_flat = y.flatten() 
z_flat = z.flatten()
dbzh_flat = dbzh_vals.flatten()

# Remove invalid data
valid = ~np.isnan(dbzh_flat)
x_valid = x_flat[valid]
y_valid = y_flat[valid]
z_valid = z_flat[valid]
dbzh_valid = dbzh_flat[valid]

# Create 3D plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Radar data (colored by DBZH)
scatter = ax.scatter(x_valid/1000, y_valid/1000, z_valid/1000, 
                    c=dbzh_valid, cmap="seismic", vmin=vmin, vmax=vmax, 
                    s=2, alpha=0.6)

# Grid structure (wireframe - sample every N points)
step = 20  # Sample every 20th point to avoid clutter
ax.plot_wireframe(x[::step, ::step]/1000, y[::step, ::step]/1000, z[::step, ::step]/1000, 
                 color='black', alpha=0.3, linewidth=0.5)

# Labels and formatting
ax.set_xlabel('East-West (km)')
ax.set_ylabel('North-South (km)')
ax.set_zlabel('Height (km)')
ax.set_title(f'Radar Data + Grid Structure - Elevation {sweeps[0].sweep_fixed_angle.values}°')

# Set axis limits
ax.set_zlim(0, 4)

# Add colorbar
plt.colorbar(scatter, label='DBZH (dBZ)', shrink=0.8)

plt.show()

**Interpolate data**

In [ ]:
def aggregate_all_elevations(sweeps, parameter='DBZH'):
    """Aggregate data from all sweeps (elevations) into one list and convert to Cartesian coordinates."""
    all_x, all_y, all_z, all_payload = [], [], [], []
    for sweep in sweeps:
        x, y, z = spherical_to_cartesian_3D(sweep)
        payload = sweep[parameter].values
        all_x.extend(x.flatten())
        all_y.extend(y.flatten())
        all_z.extend(z.flatten())
        all_payload.extend(payload.flatten())

    return np.array(all_x), np.array(all_y), np.array(all_z), np.array(all_payload)

all_x, all_y, all_z, all_dbzh = aggregate_all_elevations(sweeps, parameter='DBZH')


In [ ]:
# Load the predefined grid
grid = np.load("../data/radar_hurum_grid_10x10_8km_spacing.npz")
x_m = grid['x_centers_m']
y_m = grid['y_centers_m']
z_m = grid['z_levels_m']

grid.files

In [ ]:
# Interpolate the data at the grid center points
from scipy.interpolate import griddata

# Create 3D grid
x_grid, y_grid, z_grid = np.meshgrid(x_m, y_m, z_m)

# Only use valid data
valid = ~np.isnan(all_dbzh)
x_valid = all_x[valid]
y_valid = all_y[valid]
z_valid = all_z[valid]
dbzh_valid = all_dbzh[valid]

# Do the interpolation
grid_values = griddata(points=np.column_stack([x_valid, y_valid, z_valid]),
                       values=dbzh_valid,
                       xi=np.column_stack([x_grid.flatten(), y_grid.flatten(), z_grid.flatten()]),
                       method="nearest")

dbzh_interpolated = grid_values.reshape(x_grid.shape)


In [ ]:
# Visualize altitude slices
z_index = [0, 4, 8, 12, 16, 19]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, z in enumerate(z_index):
    axes[i].imshow(dbzh_interpolated[:,:,z], 
                   cmap="seismic", vmin=-60, vmax=60, 
                   origin="lower",
                   extent=[x_m.min()/1000, x_m.max()/1000,  # x-range (East-West)
                              y_m.min()/1000, y_m.max()/1000]) 
    axes[i].set_title(f"Altitude: {z_m[z]}m")


In [ ]:
# Plot and save all height profiles
for i, z in enumerate(z_m):
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(dbzh_interpolated[:,:,i],
              cmap="seismic", vmin=-60, vmax=60, 
              origin="lower",
              extent=[x_m.min()/1000, x_m.max()/1000,  # x-range (East-West)
              y_m.min()/1000, y_m.max()/1000])
    ax.set_title(f"Altitude: {z_m[i]}m")
    cbar = plt.colorbar(im, ax=ax)  
    cbar.set_label('DBZH (dBZ)')
    plt.savefig(f"../outputs/interpolated_data/altitude_{z_m[i]}m.png")
    plt.close();


In [ ]:
# Compare the top view of lowest elevation sweep and the lowest altitude interpolation
x, y, z = spherical_to_cartesian_3D(sweeps[0])

# Plot top view to verify correctness
fig, ax = plt.subplots(1, 2, figsize=(15, 6))  
# Top view (X-Y plane)
scatter1 = ax[0].scatter(x.flatten()/1000, y.flatten()/1000,
                     c=sweeps[0].DBZH.values.flatten(), s=0.5, cmap="seismic", vmin=vmin, vmax=vmax)
ax[0].set_xlabel('X (km)')
ax[0].set_ylabel('Y (km)')
ax[0].set_title('Lowest elevation cone top view')
ax[0].set_aspect('equal')
ax[0].set_xlim([-40,40])
ax[0].set_ylim([-40, 40])

ax[1].imshow(dbzh_interpolated[:,:,0], 
                   cmap="seismic", vmin=-60, vmax=60, 
                   origin="lower",
                   extent=[x_m.min()/1000, x_m.max()/1000,  # x-range (East-West)
                              y_m.min()/1000, y_m.max()/1000])
ax[1].set_title("Interpolation at lowest altitue")
plt.savefig("../outputs/comparison.png")

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
scatter1 = ax.scatter(x.flatten()/1000, y.flatten()/1000,
                     c=sweeps[0].DBZH.values.flatten(), s=0.5, cmap="seismic", vmin=vmin, vmax=vmax)
ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
ax.set_title('Lowest elevation cone top view')
ax.set_aspect('equal')
ax.set_xlim([-40,40])
ax.set_ylim([-40, 40])
plt.savefig("../outputs/comp_cone_top_view.png")
plt.show()

fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.imshow(dbzh_interpolated[:,:,0], 
                   cmap="seismic", vmin=-60, vmax=60, 
                   origin="lower",
                   extent=[x_m.min()/1000, x_m.max()/1000,  # x-range (East-West)
                              y_m.min()/1000, y_m.max()/1000])
ax.set_title("Interpolation at lowest altitue")
plt.savefig("../outputs/comp_interpolation_low_altitude.png")
plt.show()
